In [ ]:
#install dependencies
!pip install bitsandbytes peft datasets accelerate


In [ ]:
#establish huggingface session
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#import libraries
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

In [ ]:
#setup model

#load model from huggingface
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1", device_map="auto", quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

#freeze model weights
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)

model.lm_head= CastOutputToFloat(model.lm_head)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
config = LoraConfig(task_type="CAUSAL_LM")
model = get_peft_model(model, config)

In [1]:
#Example finetuning dataset
#Example finetuning dataset
from datasets import load_from_disk
dataset=load_from_disk("disk/")
data = data.map(lambda samples:tokenizer(samples["text"]), batched=True)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
#model training
data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)

args=TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    max_steps=200,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs"
)


trainer = Trainer(
    model = model,
    train_dataset=data["train"],
    args = args,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
1,4.022700
2,3.002300
3,3.120400
4,3.374700
5,3.680100
6,4.068900
7,3.520600
8,3.326800
9,3.353500
10,3.480100


TrainOutput(global_step=200, training_loss=2.4720520836114885, metrics={'train_runtime': 631.4579, 'train_samples_per_second': 1.267, 'train_steps_per_second': 0.317, 'total_flos': 2294207950848000.0, 'train_loss': 2.4720520836114885, 'epoch': 0.32})

In [ ]:
model.save_pretrained(".")

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "bigscience/bloom-7b1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)
print(f"Starting to load the model {model_name} into memory")

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1", quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Starting to load the model bigscience/bloom-7b1 into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
adapters_name = "."
model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()



print(f"Successfully loaded the model {model_name} into memory")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Successfully loaded the model bigscience/bloom-7b1 into memory
